In [1]:
import os
from pathlib import Path
import subprocess
import json

from notion_client import Client

from config import (
    CANDIDATE_LABELS,
    DATABASE_ID,
    NOTION_ACCESS_TOKEN,
    GYAZO_ACCESS_TOKEN,
    CACHE_PATH,
)
from lib import get_web_content, label_text, post_to_notion

/home/workspace/notion-auto-archive/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
URL = "https://arxiv.org/abs/2301.02111"

In [3]:
notion = Client(auth=NOTION_ACCESS_TOKEN)

In [4]:
p = get_web_content(url=URL, cache_path=CACHE_PATH)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  817k  100  817k    0     0   152k      0  0:00:05  0:00:05 --:--:--  171k


Downloaded 2301.02111 into /home/workspace/notion-auto-archive/content



(Loading /usr/local/share/perl/5.28.1/LaTeXML/Package/TeX.pool.ltxml...
(Loading /usr/local/share/perl/5.28.1/LaTeXML/Package/eTeX.pool.ltxml... 0.00 sec)
(Loading /usr/local/share/perl/5.28.1/LaTeXML/Package/pdfTeX.pool.ltxml... 0.00 sec) 0.08 sec)
(Loading /app/latexml/engrafo.ltxml... 0.00 sec)
(Loading /usr/src/latexml/lib/LaTeXML/Package/hyperref.sty.ltxml...
(Loading /usr/local/share/perl/5.28.1/LaTeXML/Package/url.sty.ltxml... 0.00 sec)
(Loading /usr/local/share/perl/5.28.1/LaTeXML/Package/nameref.sty.ltxml... 0.00 sec) 0.04 sec)

latexmlc (LaTeXML version 0.8.4)
invoked as [/usr/local/bin/latexmlc --format html5 --nodefaultresources --mathtex --svg --verbose --timestamp 0 --path /app/latexml/packages/ --preload /app/latexml/engrafo.ltxml --preload /usr/src/latexml/lib/LaTeXML/Package/hyperref.sty.ltxml --xsltparameter SIMPLIFY_HTML:true --css /app/dist/css/index.css --javascript /app/dist/javascript/index.js neurips_2022.tex --dest /app/output/index.html]
processing started Sa

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
QSta

In [5]:
tags = label_text(p.cleansed_content, CANDIDATE_LABELS, 0.45)

In [6]:
# import pickle
# pickle.dump(p, open("p.pkl", "wb"))

# import pickle
# p = pickle.loads(open("p.pkl", "rb").read())

In [7]:
post_to_notion(
    notion_access_token=NOTION_ACCESS_TOKEN,
    gyazo_access_token=GYAZO_ACCESS_TOKEN,
    database_id=DATABASE_ID,
    processed_content=p,
    tags=tags,
    url=URL,
)

Uploading blocks to notion...: 100%|██████████| 82/82 [00:58<00:00,  1.39it/s]


In [8]:
from markdownify import markdownify

In [9]:
from bs4 import BeautifulSoup

In [10]:
soup = BeautifulSoup(p.html_content, "html.parser")

In [11]:
ta = soup.find_all("img")

for t in ta:
    if t["src"].startswith("insert"):
        t.replace_with(f"<a>{t['src']}</a>")

In [12]:
with open(CACHE_PATH / "content.md", "w") as f:
    f.write(markdownify(soup.text))